In [12]:
import os
import shutil

In [20]:
import os.path as op

In [1]:
raw_data_path = '/Users/zeynepenkavi/Downloads/overtrained_decisions_rawfmri'

bids_path = '/Users/zeynepenkavi/Downloads/overtrained_decisions_bidsfmri'

subnums = ['601', '609', '611', '619', '621', '629']

sessions = ['01', '02', '03']

data_types = ['anat', 'func']

In [6]:
def make_bids_dirs(bids_path_ = bids_path, subnums_ = subnums, sessions_ = sessions, data_types_ = data_types):
    for cur_sub in subnums_:
        for cur_ses in sessions_:
            for cur_dt in data_types_:
                cur_dir = os.path.join(bids_path, 'sub-'+cur_sub, 'ses-'+cur_ses, cur_dt)
                if not (os.path.exists(cur_dir)):
                    os.makedirs(cur_dir)

In [7]:
make_bids_dirs()

In [9]:
misc_files = ['dataset_description.json', 'participants.json', 'README', 'CHANGES', 'LICENSE']

In [10]:
def add_misc_bids_files(bids_path_ = bids_path, misc_files_ = misc_files):
    for cur_mf in misc_files_:
        cur_template = os.path.join(os.getcwd(), 'bids_templates', cur_mf)
        cur_fn = os.path.join(bids_path, cur_mf)
        shutil.copy(cur_template, cur_fn)
    print("Don't forget to create participants.tsv!")

In [13]:
add_misc_bids_files()

Don't forget to create participants.tsv!


In [25]:
par = raw_data_path + '/sub-601/ses-01/sn_08092021_131300_5_1_fmri_run2_split.par'


In [26]:
info = dict()

In [27]:
with open(par, 'r') as f:
        lines = f.readlines()

In [28]:
found = False
for line in lines:
    found = 'Max. number of slices/locations' in line
    if found:
        info['n_slices'] = int(line.split(':')[-1].strip().replace('\n', ''))
        break

In [29]:
info

{'n_slices': 42}

In [30]:
found = False
for line_nr_of_dyns, line in enumerate(lines):
    found = 'Max. number of dynamics' in line
    if found:
        info['n_dyns'] = int(line.split(':')[-1].strip().replace('\n', ''))
        break

In [31]:
info

{'n_slices': 42, 'n_dyns': 700}

In [32]:
found = False
for line in lines:
    found = 'Max. number of echoes' in line
    if found:
        info['n_echoes'] = int(line.split(':')[-1].strip().replace('\n', ''))
        break

In [33]:
info

{'n_slices': 42, 'n_dyns': 700, 'n_echoes': 1}

In [34]:
info['n_vols'] = int(info['n_dyns'] * info['n_echoes'])

found = False
for idx_start_slices, line in enumerate(lines):
    found = '# === IMAGE INFORMATION =' in line
    if found:
        idx_start_slices += 3
        break

In [35]:
info

{'n_slices': 42, 'n_dyns': 700, 'n_echoes': 1, 'n_vols': 700}

In [36]:
idx_start_slices

101

In [37]:
idx_stop_slices = len(lines) - 2
slices = lines[idx_start_slices:idx_stop_slices]
actual_n_vols = len(slices) / info['n_slices']

In [38]:
idx_stop_slices

13961

In [39]:
actual_n_vols

330.0

In [40]:
actual_n_vols != info['n_vols']

True

In [41]:
print("Found %.3f vols (%i slices) for file %s, but expected %i dyns (%i slices);"
          " going to try to fix it by removing slices from the PAR header ..." %
          (actual_n_vols, len(slices), op.basename(par), info['n_vols'], info['n_vols']*info['n_slices']))

Found 330.000 vols (13860 slices) for file sn_08092021_131300_5_1_fmri_run2_split.par, but expected 700 dyns (29400 slices); going to try to fix it by removing slices from the PAR header ...


In [42]:
lines_to_remove = len(slices) % info['n_slices']

In [43]:
lines_to_remove

0

In [45]:
actual_n_dyns = actual_n_vols

In [46]:
actual_n_dyns

330.0

In [47]:
lines[line_nr_of_dyns] = lines[line_nr_of_dyns].replace(str(info['n_dyns']),str(int(actual_n_dyns)))

In [48]:
lines[line_nr_of_dyns]

'.    Max. number of dynamics            :   330\n'